## 交替最小二乘法优化

### 原理推导

**损失函数**
$$
J(\theta)=\sum_{u,i\in R}(r_{ui}-\mu-b_u-b_i)^2+\lambda*(\sum_u b_u{^2}+\sum_i b_i{^2} )
$$

**对损失函数求偏导**
$$
\frac{\partial{J(\theta)}}{\partial{b_u}}=-2\sum_{u,i\in R}(r_{ui}-\mu-b_u-b_i)+2\lambda * b_u
$$
**另偏导等于0，可得：**
$$
\sum_{u,i\in R}(r_{ui}-\mu-b_u-b_i)=\lambda * b_u \\
\sum_{u,i\in R}(r_{ui}-\mu-b_i)=\sum_{u,i\in R} b_u+\lambda * b_u
$$
为了**简化公式**，令$\sum_{u,i\in R} b_u \approx |R(u)|*b_u$，$|R(u)|$是$u$总的评分次数，可得：
$$
b_u=\frac{\sum_{u,i\in R}(r_{ui}-\mu-b_i)}{|R(u)|+\lambda_1}
$$
**同理可得：**
$$
b_i=\frac{\sum_{u,i\in R}(r_{ui}-\mu-b_u)}{|R(i)|+\lambda_2}
$$
其中$|R(i)|$是$i$总的被评分次数

通过原理推导，我们得到了$b_u$和$b_i$的表达式，他们的表达式中又各自包含着对方，可以用交替最小二乘的方法来计算他们的值。



In [26]:
import numpy as np
import pandas as pd

In [28]:
dataset = pd.read_csv("../dataset/ml-latest-small/ratings.csv",dtype={"userId":np.int32, "movieId":np.int32,"rating":np.float32},usecols=range(3))
dataset = pd.DataFrame(dataset)

In [40]:
class BaselineCFBySGD(object):
    def __init__(self, number_epochs=10, lambda1=0.1, lambda2=0.1, columns: list=["userId","movieId","rating"]):
        # 梯度下降最高迭代次数
        self.number_epochs = number_epochs
        # 正则参数
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        # 数据集字段名称
        self.columns = columns

    def fit(self, dataset):
        """
        训练
        :param dataset: uid, iid, rating
        :return:
        """
        self.dataset = dataset
        # 用户评分数据
        self.user_ratings = self.dataset.groupby(self.columns[0]).agg([list])[[self.columns[1], self.columns[2]]]
        # 物品评分数据
        self.item_ratings = self.dataset.groupby(self.columns[1]).agg([list])[[self.columns[0], self.columns[2]]]
        # 计算全局平均分
        self.global_mean = self.dataset[self.columns[2]].mean()
#         # 计算各用户评分次数
#         self.R_u = self.dataset.groupby(self.columns[0]).count()[self.columns[1]]
#         # 计算各物品被评分次数
#         self.R_i = self.dataset.groupby(self.columns[1]).count()[self.columns[0]]

        # 调用als方法训练模型参数
        self.bu, self.bi = self.als()

    def als(self):
        """
        随机梯度下降优化bu, bi
        :return: bu, bi
        """
        # 初始化参数 bu, bi  全部设置为0
        bu = dict(zip(self.user_ratings.index, np.zeros(len(self.user_ratings.index))))
        bi = dict(zip(self.item_ratings.index, np.zeros(len(self.item_ratings.index))))

        # 交替最小二乘法更新参数
        for epoch in range(self.number_epochs):
            print("Epoch: %d" % epoch)

            for uid, iid_list, real_rating_list in self.user_ratings.itertuples(index=True):
                sum = 0
                for iid, real_rating in zip(iid_list, real_rating_list):
                    sum += real_rating - self.global_mean - bi[iid]
                bu[uid] = sum / (self.lambda1 * len(iid_list))
#                 bu[uid] = sum / (self.lambda1 * self.R_u.loc[uid])

            for iid, uid_list, real_rating_list in self.item_ratings.itertuples(index=True):
                sum = 0
                for uid, real_rating in zip(uid_list, real_rating_list):
                    sum += real_rating - self.global_mean - bu[uid]
                bi[iid] = sum / (self.lambda2 * len(uid_list))
#                 bi[iid] = sum / (self.lambda2 * self.R_i.loc[iid])
    
        return bu, bi

    def predict(self, uid, iid):
        """预测评分"""
        predict_rating = self.global_mean + self.bu[uid] + self.bi[iid]
        return predict_rating

    def test(self, testset):
        """预测测试集数据"""
        for uid, iid, real_rating in testset.itertuples(index=False):
            try:
                pred_rating = self.predict(uid, iid)
            except Exception as e:
                print(e,"出现异常~")
            else:
                yield uid, iid, real_rating, pred_rating

In [41]:
model = BaselineCFBySGD(number_epochs=15)
model.fit(dataset)

Epoch: 0
Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 5
Epoch: 6
Epoch: 7
Epoch: 8
Epoch: 9
Epoch: 10
Epoch: 11
Epoch: 12
Epoch: 13
Epoch: 14


In [47]:
model.predict(1,1)

-1.1524593008421473e+23

疑惑的点：我不知道为什么在交替更新bu和bi参数的时候，先后顺序会对预测的结果有影响，也就是将两个for循环颠倒一下位置结果就不一样了,视频中是这样实现的（先根据bu去更新bi,然后根据更新好的bi去更新bu）：
```python
            for iid, uid_list, real_rating_list in self.item_ratings.itertuples(index=True):
                sum = 0
                for uid, real_rating in zip(uid_list, real_rating_list):
                    sum += real_rating - self.global_mean - bu[uid]
                bi[iid] = sum / (self.lambda2 * len(uid_list))            
            for uid, iid_list, real_rating_list in self.user_ratings.itertuples(index=True):
                sum = 0
                for iid, real_rating in zip(iid_list, real_rating_list):
                    sum += real_rating - self.global_mean - bi[iid]
                bu[uid] = sum / (self.lambda1 * len(iid_list))        
```